## People Centered COS

### Explore geography columns - Latitude, Longitude, City, County 

#### Analysis Questions
  
- How often does the city align (or not align) with the county?
   - The cities seem to be aligned with the correct county. This is a bit tricky for cities that are associated with multiple counties. 
- How often does the latitude and longitude align (or not align) with the city?
   - Spot checking, yes, it seems the lat and long often align with the city.
   - However, we know of that case with the Denver lat long value being associated with Colorado Springs. 
- Can the missing city values be populated by the latitude and longitude values?
   - Yes, this seems worth while. 
 
- Rough bounds of colorado state for latitude and longitude
  - 37, -102 (south east corner)
  - 40.1, -108.9 (north west corner 

- Research Mapping Tools 
    - Map box and geocoding 

### Set Up 

In [1]:
pip install geocoder


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geopy


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install plotly


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install openpyxl


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import geocoder

from geopy.geocoders import Nominatim

In [7]:
pd.set_option('display.max_rows', 500)

### Load CDOT Data from 2020 - 2024

In [8]:
import sys  
sys.path.insert(1, '../')

from convert_old_cdot_format_to_new_format import convert_old_cdot_format_to_new_format

In [9]:
# Load CDOT Data for a list of years

years = [2024]

cdot_base_path = "/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/"

loaded_data_list = []
for y in years:
    filename = f"CDOTRM_CD_Crash_Listing_-_{y}.xlsx"
    full_path = cdot_base_path + filename
    print(full_path)
    year_cdot_pdf = pd.read_excel(cdot_base_path + filename)
    year_cdot_pdf['Crash Year'] = y
    print(list(year_cdot_pdf.columns)[0:5])
    print(year_cdot_pdf.shape)
    
    if y < 2021:
        # call conversion code 
        year_cdot_pdf = convert_old_cdot_format_to_new_format(year_cdot_pdf)

    loaded_data_list.append(year_cdot_pdf) 
    
cdot_pdf_load = pd.concat(loaded_data_list)

/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2024.xlsx
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(92810, 111)


### Clean Data

In [10]:
print('Loaded Data Size', cdot_pdf_load.shape)

# Drop duplicates
cdot_pdf_withNans_dedup = cdot_pdf_load.drop_duplicates()
print('Data size after drop duplicates', cdot_pdf_withNans_dedup.shape)

display(cdot_pdf_withNans_dedup.head(5))

Loaded Data Size (92810, 111)
Data size after drop duplicates (92810, 111)


,CUID,System Code,Rd_Number,Rd_Section,City_Street,Crash Date,Crash Time,Agency Id,City,County,...,TU-1 NM Alcohol Suspected,TU-2 NM Alcohol Suspected,TU-1 NM Marijuana Suspected,TU-2 NM Marijuana Suspected,TU-1 NM Other Drugs Suspected,TU-2 NM Other Drugs Suspected,Record Status,Processing Status,Last Updated,Crash Year
0,453608,City Street,0320,SANTA,THAVE,2024-01-01,00:16:00,DPD,DENVER,DENVER,...,NaN,NaN,NaN,NaN,NaN,NaN,Coded,Completed,Jun 13 2024 2:21PM,2024
1,420975,State Highway,391A,8.482,NaN,2024-01-01,00:34:00,WRPD,WHEAT RIDGE,JEFFERSON,...,NaN,NaN,NaN,NaN,NaN,NaN,Coded,Completed,Feb 26 2024 10:46PM,2024
2,466850,State Highway,093A,16.461,NaN,2024-01-01,00:34:00,BLDPD,BOULDER,BOULDER,...,NaN,NaN,NaN,NaN,NaN,NaN,Coded,Completed,Jun 27 2024 12:21AM,2024
3,453551,City Street,1285,THAVE,COLOR,2024-01-01,00:37:00,THPD,THORNTON,ADAMS,...,NaN,NaN,NaN,NaN,NaN,NaN,Coded,Completed,Jun 13 2024 2:20PM,2024
4,467394,State Highway,082A,29.988,NaN,2024-01-01,00:40:00,CSP,NaN,PITKIN,...,NaN,NaN,NaN,NaN,NaN,NaN,Coded,Completed,Jun 19 2024 1:03PM,2024


In [11]:
print(list(cdot_pdf_withNans_dedup.columns))

['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street', 'Crash Date', 'Crash Time', 'Agency Id', 'City', 'County', 'Latitude', 'Longitude', 'Location 1', 'Link', 'Location 2', 'Location', 'Road Description', 'First HE', 'Second HE', 'Third HE', 'Fourth HE', 'MHE', 'Crash Type', 'Approach Overtaking Turn', 'Wild Animal', 'Number Killed', 'Number Injured', 'Injury 00', 'Injury 01', 'Injury 02', 'Injury 03', 'Injury 04', 'Total Vehicles', 'Secondary Crash', 'Construction Zone', 'School Zone', 'Road Contour Curves', 'Road Contour Grade', 'Road Condition', 'Lighting Conditions', 'Weather Condition', 'Weather Condition 2', 'Lane Position', 'TU-1 Direction', 'TU-2 Direction', 'TU-1 Movement', 'TU-2 Movement', 'TU-1 Type', 'TU-2 Type', 'TU-1 Special Function', 'TU-2 Special Function', 'TU-1 Autonomous Vehicle', 'TU-2 Autonomous Vehicle', 'TU-1 Hit And Run', 'TU-2 Hit And Run', 'TU-1 Speed Limit', 'TU-2 Speed Limit', 'TU-1 Estimated Speed', 'TU-2 Estimated Speed', 'TU-1 Speed', 'TU-2 

In [12]:
# Select columns for analysis 
analysis_columns = [
    'Crash Date', 
    'City', 
    'County',
    'Crash Year',
    'Latitude', 
    'Longitude',
]

cdot_pdf_select_columns = cdot_pdf_withNans_dedup[analysis_columns]

# check NaN count by column 
for col in cdot_pdf_select_columns.columns:
    print(col, cdot_pdf_select_columns.loc[cdot_pdf_select_columns[col].isna()].shape[0])

Crash Date 0
City 24114
County 68
Crash Year 0
Latitude 5635
Longitude 5633


In [13]:
cdot_pdf_select_columns.describe()

,Crash Date,Crash Year,Latitude,Longitude
count,92810,92810.0,87175.000000,87177.000000
mean,2024-06-21 17:43:38.329921536,2024.0,39.687484,-105.044129
min,2024-01-01 00:00:00,2024.0,37.000510,-122.465870
25%,2024-03-22 00:00:00,2024.0,39.374380,-105.093300
50%,2024-06-20 00:00:00,2024.0,39.725500,-104.983880
75%,2024-09-20 00:00:00,2024.0,39.885490,-104.822510
max,2024-12-31 00:00:00,2024.0,999.999990,999.999990
std,NaN,0.0,10.822980,12.446675


In [77]:
# fill null values that screw up counting
cdot_pdf = cdot_pdf_select_columns.copy()
    
# Fill NaN values 
cdot_pdf['Crash Date'] = cdot_pdf['Crash Date'].fillna('no crash date provided')
cdot_pdf['Crash Year'] = cdot_pdf['Crash Year'].fillna('no crash date provided')
cdot_pdf['City'] = cdot_pdf['City'].fillna('no city provided')
cdot_pdf['County'] = cdot_pdf['County'].fillna('no county provided')
cdot_pdf['Latitude'] = cdot_pdf['Latitude'].fillna(999.999000)
cdot_pdf['Longitude'] = cdot_pdf['Longitude'].fillna(999.999000)

# mark "unknown" city as 'no city provided'
cdot_pdf.loc[cdot_pdf['City'].str.contains('UNKNOWN'), 'City'] = 'no city provided'

In [78]:
# check NaN count by column 
for col in cdot_pdf.columns:
    print(col, cdot_pdf.loc[cdot_pdf[col].isna()].shape[0])

Crash Date 0
City 0
County 0
Crash Year 0
Latitude 0
Longitude 0


In [79]:
display(cdot_pdf.head(10))

,Crash Date,City,County,Crash Year,Latitude,Longitude
0,2024-01-01,DENVER,DENVER,2024,39.73687,-104.99869
1,2024-01-01,WHEAT RIDGE,JEFFERSON,2024,39.76933,-105.10966
2,2024-01-01,BOULDER,BOULDER,2024,39.98818,-105.25394
3,2024-01-01,THORNTON,ADAMS,2024,39.92111,-104.94027
4,2024-01-01,no city provided,PITKIN,2024,39.30934,-106.93615
5,2024-01-01,no city provided,WELD,2024,40.96810,-104.79158
6,2024-01-01,AURORA,ARAPAHOE,2024,999.99900,999.99900
7,2024-01-01,COLORADO SPRINGS,EL PASO,2024,38.96059,-104.78723
8,2024-01-01,FORT COLLINS,LARIMER,2024,40.58422,-105.07556
9,2024-01-01,DENVER,DENVER,2024,39.70743,-104.86597


### Check Mapping from City to County

In [80]:
print(f"Total crashes in {years}:", cdot_pdf.shape[0]) 

Total crashes in [2024]: 92810


In [81]:
# how many County values are not defined? 
print('crashes missing County info', cdot_pdf.loc[cdot_pdf['County'] == 'no county provided', 'County'].count())
print()

# how many City values are not defined? 
print('crashes missing City info', cdot_pdf.loc[cdot_pdf['City'] == 'no city provided', 'City'].count())
print()

crashes missing County info 68

crashes missing City info 24114



In [82]:
# remove missing county and missing city info 
cdot_pdf_filt = cdot_pdf.loc[(cdot_pdf['County'] != 'no county provided') & (cdot_pdf['City'] != 'no city provided')]

print(cdot_pdf_filt.shape)

(68696, 6)


In [83]:
# Get the expected county for each City in dataset and save to csv

#city_to_county_pdf = pd.DataFrame(columns=['City', 'County']) 
#city_to_county_pdf['City'] = cdot_pdf_filt['City'].unique()
#display(city_to_county_pdf)
#
#geolocator = Nominatim(user_agent="city_county_locator")
#for row_num, row_dict in city_to_county_pdf.iterrows():
#    print(row_dict['City']) 
#
#    try:
#        location = geolocator.geocode(f"{row_dict['City'].lower()} Colorado",language='en')
#        print(location.raw['display_name'])
#        # get county
#        county = location.raw['display_name'].rsplit(',')[1].replace('County', '').upper().strip(' ')
#        if county == 'COLORADO':
#            county = row_dict['City'].upper()
#        print(county)
#        print()
#        # set county value
#        city_to_county_pdf.loc[city_to_county_pdf['City'] == row_dict['City'], 'County'] = county 
#    except:
#        print('api error')
#
# save mapping out to csv file so we don't need to generate again ... 
#city_to_county_pdf.to_csv('/Users/haley.northrup/Desktop/PeopleCenteredCOS/colorado_city_to_county_mapping.csv', index=False)


In [84]:
# Load CSV
city_to_county_pdf = pd.read_csv('/Users/haley.northrup/Desktop/PeopleCenteredCOS/colorado_city_to_county_mapping.csv')
display(city_to_county_pdf.head(5))

,City,County
0,GYPSUM,EAGLE
1,CENTENNIAL,ARAPAHOE
2,DENVER,DENVER
3,THORNTON,ADAMS
4,PUEBLO,PUEBLO


In [85]:
# Now we have a mapping of city to expected County 
# let's check how many are wrong 
city_to_county_pdf = city_to_county_pdf.rename(columns={'County':'Expected_County'}) 

joined_df = cdot_pdf_filt.merge(city_to_county_pdf, on='City') 
display(joined_df.head(10))

,Crash Date,City,County,Crash Year,Latitude,Longitude,Expected_County
0,2024-01-01,DENVER,DENVER,2024,39.736870,-104.998690,DENVER
1,2024-01-01,WHEAT RIDGE,JEFFERSON,2024,39.769330,-105.109660,JEFFERSON
2,2024-01-01,BOULDER,BOULDER,2024,39.988180,-105.253940,BOULDER
3,2024-01-01,THORNTON,ADAMS,2024,39.921110,-104.940270,ADAMS
4,2024-01-01,AURORA,ARAPAHOE,2024,999.999000,999.999000,ADAMS
5,2024-01-01,COLORADO SPRINGS,EL PASO,2024,38.960590,-104.787230,EL PASO
6,2024-01-01,FORT COLLINS,LARIMER,2024,40.584220,-105.075560,LARIMER
7,2024-01-01,DENVER,DENVER,2024,39.707430,-104.865970,DENVER
8,2024-01-01,LONGMONT,BOULDER,2024,40.153380,-105.093090,BOULDER
9,2024-01-01,BROOMFIELD,BROOMFIELD,2024,39.922379,-105.090823,BROOMFIELD


In [86]:
joined_df['county_is_correct'] = joined_df['County'] == joined_df['Expected_County']
display(joined_df.head(5))

print('Count of crashes where county doesnt match expected', (joined_df['county_is_correct'] != True).sum())
#display(joined_df.loc[joined_df['county_is_correct'] != True].head(100))

# however some cities are in multiple counties 
condition = (joined_df['county_is_correct'] != True) & ~(joined_df['City'].isin(['AURORA', 'WESTMINSTER', 'LITTLETON', 'JOHNSTOWN', 'ARVADA', 'ERIE']))

print('after removing cities with multiple counties', joined_df.loc[condition].shape)

display(joined_df.loc[condition].head(100))

,Crash Date,City,County,Crash Year,Latitude,Longitude,Expected_County,county_is_correct
0,2024-01-01,DENVER,DENVER,2024,39.73687,-104.99869,DENVER,True
1,2024-01-01,WHEAT RIDGE,JEFFERSON,2024,39.76933,-105.10966,JEFFERSON,True
2,2024-01-01,BOULDER,BOULDER,2024,39.98818,-105.25394,BOULDER,True
3,2024-01-01,THORNTON,ADAMS,2024,39.92111,-104.94027,ADAMS,True
4,2024-01-01,AURORA,ARAPAHOE,2024,999.99900,999.99900,ADAMS,False


Count of crashes where county doesnt match expected 5708
after removing cities with multiple counties (302, 8)


,Crash Date,City,County,Crash Year,Latitude,Longitude,Expected_County,county_is_correct
377,2024-01-03,BASALT,PITKIN,2024,39.359590,-107.027710,EAGLE,False
807,2024-01-05,FORT CARSON,LARIMER,2024,40.581430,-105.064760,EL PASO,False
969,2024-01-06,LONGMONT,WELD,2024,40.160660,-105.036880,BOULDER,False
974,2024-01-06,PLATTEVILLE,DOUGLAS,2024,40.214950,-104.821140,WELD,False
1123,2024-01-07,TWIN LAKES,ADAMS,2024,39.493900,-105.002250,LAKE,False
1416,2024-01-08,LAKEWOOD,DENVER,2024,39.682310,-105.053180,JEFFERSON,False
1560,2024-01-09,LAKEWOOD,DENVER,2024,39.667460,-105.081610,JEFFERSON,False
1948,2024-01-11,MOUNTAIN VIEW,DENVER,2024,999.999000,999.999000,JEFFERSON,False
1991,2024-01-11,DENVER,ADAMS,2024,39.831870,-105.006240,DENVER,False
2886,2024-01-15,LAKEWOOD,WELD,2024,40.350360,-104.704120,JEFFERSON,False


## Check Latitude and Longitude Mapping to City 

In [91]:
# remove missing Latitude and Longitude info
cdot_pdf_w_lat_long = cdot_pdf.loc[~(cdot_pdf['Latitude'] == 999.999000) & ~(cdot_pdf['Longitude'] == 999.999000)]

print(cdot_pdf_w_lat_long.shape)

(87165, 6)


In [92]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

def get_city_from_coordinates(latitude, longitude):
    """
    Retrieves the city and address details from given latitude and longitude.
    """
    geolocator = Nominatim(user_agent="hmn") # Replace "my_geocoding_app" with a unique identifier
    try:
        location = geolocator.reverse((latitude, longitude), exactly_one=True)
        if location:
            address = location.raw.get('address', {})
            city = address.get('city') or \
                   address.get('town') or \
                   address.get('village')
            return city, location.address
        else:
            return None, None
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        print(f"Geocoding error: {e}")
        return None, None

latitude = 40.7128  # Example latitude (New York City)
longitude = -74.0060 # Example longitude (New York City)
city, full_address = get_city_from_coordinates(latitude, longitude)

print(city)
print(full_address)

City of New York
New York City Hall, 260, Broadway, Lower Manhattan, Manhattan, New York County, City of New York, New York, 10000, United States


In [110]:
small_sample = cdot_pdf_w_lat_long.head(10)

small_sample[['city_from_coords', 'full_address_from_coords']] = small_sample.apply(
        lambda row: get_city_from_coordinates(row['Latitude'], row['Longitude']),
        axis=1,
        result_type='expand'
    )

display(small_sample[['Latitude', 'Longitude', 'City', 'city_from_coords', 'full_address_from_coords']])

/var/folders/wh/mh9pm3896vb556vt9fxjp6sc0000gq/T/ipykernel_15144/354971835.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_sample[['city_from_coords', 'full_address_from_coords']] = small_sample.apply(
/var/folders/wh/mh9pm3896vb556vt9fxjp6sc0000gq/T/ipykernel_15144/354971835.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_sample[['city_from_coords', 'full_address_from_coords']] = small_sample.apply(


,Latitude,Longitude,City,city_from_coords,full_address_from_coords
0,39.73687,-104.99869,DENVER,Denver,"816, West 13th Avenue, Mariposa District, Linc..."
1,39.76933,-105.10966,WHEAT RIDGE,Wheat Ridge,"9999, West 38th Avenue, Wheat Ridge, Jefferson..."
2,39.98818,-105.25394,BOULDER,Boulder,"South Broadway, Martin Acres, Boulder, Boulder..."
3,39.92111,-104.94027,THORNTON,Thornton,"East 124th Avenue, Thornton, Adams County, Col..."
4,39.30934,-106.93615,no city provided,None,"State Highway 82, Pitkin County, Colorado, Uni..."
5,40.96810,-104.79158,no city provided,None,"65992, United States Highway 85, Weld County, ..."
7,38.96059,-104.78723,COLORADO SPRINGS,Colorado Springs,"1976, Research Parkway, Colorado Springs, El P..."
8,40.58422,-105.07556,FORT COLLINS,Fort Collins,"199, East Olive Street, Fort Collins, Larimer ..."
9,39.70743,-104.86597,DENVER,Denver,"South Havana Street, Denver, Colorado, 80247, ..."
10,40.15338,-105.09309,LONGMONT,Longmont,"Longmont, Boulder County, Colorado, 80501, Uni..."


In [109]:
# filter to where city is not provided and see how often we can get the associated city 
cdot_pdf_w_latlong_and_no_city = cdot_pdf_w_lat_long.loc[cdot_pdf_w_lat_long['City'] == 'no city provided']

print(cdot_pdf_w_latlong_and_no_city.size)

no_city_small_sample = cdot_pdf_w_latlong_and_no_city.head(100)

no_city_small_sample[['city_from_coords', 'full_address_from_coords']] = no_city_small_sample.apply(
        lambda row: get_city_from_coordinates(row['Latitude'], row['Longitude']),
        axis=1,
        result_type='expand'
    )

display(no_city_small_sample[['Latitude', 'Longitude', 'City', 'city_from_coords', 'full_address_from_coords']].head(5))

142092


/var/folders/wh/mh9pm3896vb556vt9fxjp6sc0000gq/T/ipykernel_15144/1278271060.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_city_small_sample[['city_from_coords', 'full_address_from_coords']] = no_city_small_sample.apply(
/var/folders/wh/mh9pm3896vb556vt9fxjp6sc0000gq/T/ipykernel_15144/1278271060.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_city_small_sample[['city_from_coords', 'full_address_from_coords']] = no_city_small_sample.apply(


,Latitude,Longitude,City,city_from_coords,full_address_from_coords
4,39.30934,-106.93615,no city provided,None,"State Highway 82, Pitkin County, Colorado, Uni..."
5,40.96810,-104.79158,no city provided,None,"65992, United States Highway 85, Weld County, ..."
14,38.51459,-104.62895,no city provided,None,"5503, John F. Kennedy Memorial Highway, Pueblo..."
27,39.88517,-104.90635,no city provided,Thornton,"6546, East 104th Avenue, Thornton, Adams Count..."
34,40.09103,-105.15231,no city provided,Niwot,"6385, Snowberry Lane, Niwot, Boulder County, C..."


In [95]:
# TOP 2 
# IS THIS THE DEFAULT VALUE FOR LAT LONG??? IT IS IN DENVER???
latitude = 39.746  
longitude = -104.989 
city, full_address = get_city_from_coordinates(latitude, longitude)

print(city)
print(full_address)

Denver
Grand Hyatt Denver, 1750, Welton Street, Central Business District, Denver, Colorado, 80202, United States


In [116]:
## Lets check the lat long to city association for El Paso County crashes
cdot_pdf_w_lat_long_elpaso = cdot_pdf_w_lat_long.loc[cdot_pdf_w_lat_long['County'] == 'EL PASO']

display(cdot_pdf_w_lat_long_elpaso.head(5))

,Crash Date,City,County,Crash Year,Latitude,Longitude
7,2024-01-01,COLORADO SPRINGS,EL PASO,2024,38.96059,-104.78723
22,2024-01-01,COLORADO SPRINGS,EL PASO,2024,38.91169,-104.70531
28,2024-01-01,COLORADO SPRINGS,EL PASO,2024,38.92485,-104.76769
29,2024-01-01,COLORADO SPRINGS,EL PASO,2024,38.92483,-104.76770
40,2024-01-01,COLORADO SPRINGS,EL PASO,2024,38.91574,-104.78199


In [117]:
cdot_pdf_w_lat_long_elpaso[['city_from_coords', 'full_address_from_coords']] = cdot_pdf_w_lat_long_elpaso.apply(
        lambda row: get_city_from_coordinates(row['Latitude'], row['Longitude']),
        axis=1,
        result_type='expand'
    )

display(cdot_pdf_w_lat_long_elpaso[['County', 'Latitude', 'Longitude', 'City', 'city_from_coords', 'full_address_from_coords']].head(5))

Geocoding error: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=38.96659&lon=-104.74589&format=json&addressdetails=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Geocoding error: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=38.81467&lon=-104.82452&format=json&addressdetails=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


/var/folders/wh/mh9pm3896vb556vt9fxjp6sc0000gq/T/ipykernel_15144/3151118852.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdot_pdf_w_lat_long_elpaso[['city_from_coords', 'full_address_from_coords']] = cdot_pdf_w_lat_long_elpaso.apply(
/var/folders/wh/mh9pm3896vb556vt9fxjp6sc0000gq/T/ipykernel_15144/3151118852.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdot_pdf_w_lat_long_elpaso[['city_from_coords', 'full_address_from_coords']] = cdot_pdf_w_lat_long_elpaso.apply(


,County,Latitude,Longitude,City,city_from_coords,full_address_from_coords
7,EL PASO,38.96059,-104.78723,COLORADO SPRINGS,Colorado Springs,"1976, Research Parkway, Colorado Springs, El P..."
22,EL PASO,38.91169,-104.70531,COLORADO SPRINGS,Colorado Springs,"Stetson Hills Boulevard, Colorado Springs, El ..."
28,EL PASO,38.92485,-104.76769,COLORADO SPRINGS,Colorado Springs,"3197, Dublin Boulevard, Colorado Springs, El P..."
29,EL PASO,38.92483,-104.76770,COLORADO SPRINGS,Colorado Springs,"6398, Flintridge Drive, Colorado Springs, El P..."
40,EL PASO,38.91574,-104.78199,COLORADO SPRINGS,Colorado Springs,"2499, Vickers Drive, Colorado Springs, El Paso..."


In [ ]:
#for index, row in cdot_pdf_w_lat_long_elpaso.iterrows():
#    print(index)
#    print(row)
#    city, full_address = get_city_from_coordinates(row['Latitude'], row['Longitude'])
#    print(city)
#    print()

In [118]:
cdot_pdf_w_lat_long_elpaso.loc[cdot_pdf_w_lat_long_elpaso['City'] == cdot_pdf_w_lat_long_elpaso['city_from_coords'].str.upper()].shape

(6613, 8)

In [119]:
#display(cdot_pdf_w_lat_long_elpaso.loc[cdot_pdf_w_lat_long_elpaso['City'] != cdot_pdf_w_lat_long_elpaso['city_from_coords'].str.upper()])

In [120]:
cdot_pdf_w_lat_long_elpaso.groupby('City').count()

,Crash Date,County,Crash Year,Latitude,Longitude,city_from_coords,full_address_from_coords
City,,,,,,,
ALLISON,1,1,1,1,1,1,1
BLACK FOREST,2,2,2,2,2,0,2
CALHAN,5,5,5,5,5,0,5
CIMARRON,2,2,2,2,2,1,2
COLORADO CITY,2,2,2,2,2,2,2
COLORADO SPRINGS,6218,6218,6218,6218,6218,6132,6216
DENVER,1,1,1,1,1,1,1
FORT LUPTON,1,1,1,1,1,0,1
FOUNTAIN,448,448,448,448,448,385,448


In [121]:
cdot_pdf_w_lat_long_elpaso.groupby('city_from_coords').count()

,Crash Date,City,County,Crash Year,Latitude,Longitude,full_address_from_coords
city_from_coords,,,,,,,
Black Forest,16,16,16,16,16,16,16
Centennial,1,1,1,1,1,1,1
Colorado Springs,6536,6536,6536,6536,6536,6536,6536
Denver,1,1,1,1,1,1,1
Divide,3,3,3,3,3,3,3
Falcon,148,148,148,148,148,148,148
Fort Carson,68,68,68,68,68,68,68
Fountain,510,510,510,510,510,510,510
Gleneagle,25,25,25,25,25,25,25
